# Predict with Vertex AI Endpoints

In [ ]:
import logging
from operator import itemgetter
import os

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import requests
import tensorflow as tf

In [ ]:
gcp_project_id = "<TO DEFINE>"
gcp_region = "europe-west1"
endpoint_id = "<TO DEFINE>"

In [ ]:
def preprocess_image(image_url):
    logging.info(f'Fetching image from URL: {image_url}')
    try:
        image_response = requests.get(image_url)
        image_response.raise_for_status()
        assert image_response.headers.get('Content-Type') == 'image/png'
    except (ConnectionError, requests.exceptions.RequestException,
            AssertionError):
        logging.error(f'Error fetching image from URL: {image_url}')
        return None

    logging.info('Decoding and preprocessing image ...')
    image = tf.io.decode_jpeg(image_response.content, channels=1)
    image = tf.image.resize_with_pad(image, 64, 64)
    return image.numpy().tolist()  # make it JSON-serializable

In [ ]:
def get_prediction(instance, aip_endpoint_name):
    logging.info('Sending prediction request to AI Platform ...')
    try:
        pb_instance = json_format.ParseDict(instance, Value())
        response = aip_client.predict(endpoint=aip_endpoint_name,
                                      instances=[pb_instance])
        return list(response.predictions[0])
    except Exception as err:
        logging.error(f'Prediction request failed: {type(err)}: {err}')
        return None

In [ ]:
aip_client = aiplatform.gapic.PredictionServiceClient(client_options={
    'api_endpoint': 'europe-west1-aiplatform.googleapis.com'
})
aip_endpoint_name = f'projects/{gcp_project_id}/locations/{gcp_region}/endpoints/{endpoint_id}'

In [ ]:
img_url = "https://storage.googleapis.com/devoxx_poc_public_images/cat/cat_6033.png"
instance = preprocess_image(img_url)
probabilities = get_prediction(instance, aip_endpoint_name)
print(probabilities)

In [ ]:
img_classes = ['angel', 'cat', 'crown', 'The Eiffel Tower', 'The Mona Lisa']

probabilities_with_classes = zip(img_classes, probabilities)
sorted_probabilities = sorted(probabilities_with_classes,
                              key=itemgetter(1),
                              reverse=True)
print(sorted_probabilities)